# Importing Dependencies

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import glob
import os
import re
import pandas as pd
import numpy as np
from transformers import GPT2TokenizerFast
from tqdm import tqdm
import time

# Hyperparameters

In [ ]:
batch_size = 2
block_size = 1024
max_iters = 5000
eval_interval = 100
learning_rate = 3e-4
eval_iters = 100
n_embd = 768
n_head = 12
n_layer = 12
dropout = 0.2

In [ ]:
gpu_count = torch.cuda.device_count()
device = []
for i in range(gpu_count):
    device.append('cuda:{}'.format(i))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# batch_size = 1
# block_size = 2048
# max_iters = 5000
# eval_interval = 0
# learning_rate = 3e-4
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# eval_iters = 0
# n_embd = 2560
# n_head = 32
# n_layer = 32
# dropout = 0.2

In [ ]:
torch.manual_seed(42)

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
vocab_size = tokenizer.vocab_size

# Dataset Pipeline

In [ ]:
books = glob.glob("D:\AIML Projects\ProdigalAI\learn gpt\*.txt")

In [ ]:
len(books)

In [ ]:
def yield_books():
    for i in books:
        with open(i, "r", encoding="utf-8") as f:
            yield f.read()

----

In [ ]:
def load_new_data(interval):
    books = glob.glob("/home/prodigalai/sam/custom-gpt/datasets/*/*.txt")
    books = books[:16000]
    books = books[interval:interval+100]
    text = ""
    for book in books:
        with open(book, 'r', encoding='utf-8') as f:
            text += f.read()
    text = preprocess_text(text)
    text = tokenizer.encode(text)
    data = torch.tensor(text, dtype=torch.long)
    n = int(0.9*len(data))
    train_data = data[:n]
    valid_data = data[n:]
    return train_data, valid_data

In [ ]:
# every 1000 epoch train data should be next 100 books
def book_batch(books):
    for i in range(0, len(books), 100):
        yield books[i:i+100]

In [ ]:
books = books[:100]

In [ ]:
text = ""
for book in tqdm(books):
   with open(book, 'r', encoding='utf-8') as f:
      text += f.read()

In [ ]:
def preprocess_text(text):
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff\xad\x0c6§\\\£\Â*_<>""⎫•{}Γ~]', ' ', text)
    #text = re.sub('[^A-Za-z0-9]+', ' ', text)
    #text = re.sub(r'\s', ' ', text)
    text = re.sub('\n', ' ', text)
    text = re.sub(r'\s((I{2,}V*X*\.*)|(IV\.*)|(IX\.*)|(V\.*)|(V+I*\.*)|(X+L*V*I*]\.*))\s', ' ', text)
    text = re.sub(r'\s((i{2,}v*x*\.*)|(iv\.*)|(ix\.*)|(v\.*)|(v+i*\.*)|(x+l*v*i*\.*))\s', ' ', text)
    text = re.sub(r'&', 'and', text)
    #text = re.sub(r'[A-Z]{2,}', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    #text = text.lower()
    return text

In [ ]:
text = preprocess_text(text)

In [ ]:
with open(r"D:\AIML Projects\ProdigalAI\implement gpt\stanford_alpaca\alpaca_data.json", "r", encoding="utf-8") as f:
    text = f.read()

In [ ]:
len(text)

In [ ]:
text[:200]

In [ ]:
vocab_size

In [ ]:
print(tokenizer.encode("hello world"))
print(tokenizer.decode(tokenizer.encode("hello world")))

In [ ]:
tensor = tokenizer.encode(text)

In [ ]:
data = torch.tensor(tensor, dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
valid_data = data[n:]

In [ ]:
def get_batch(split):
    data = train_data if split == "train" else valid_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    for i in range(len(train_data)):
        x = torch.stack([data[i:i+block_size] for i in ix])
        y = torch.stack([data[i+1:i+block_size+1] for i in ix])
        x, y = x.to(device), y.to(device)
        return x, y

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [ ]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [ ]:
print("Using", torch.cuda.device_count(), 'GPU!')
model = GPTLanguageModel()
model = nn.DataParallel(model)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.mean().item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        #train_data, valid_data = load_new_data()
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        
        #torch.save(model.state_dict(), "/kaggle/working/" + "epoch_" + str(iter) + ".pth")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.mean().backward()
    optimizer.step()

In [ ]:
m.load_state_dict(torch.load('D:\AIML Projects\ProdigalAI\implement gpt\epoch_16000.pt'))

In [ ]:
input = "allah"

In [ ]:
preds = torch.tensor([tokenizer.encode(input)], dtype=torch.long, device=device)
preds = m.module.generate(preds, max_new_tokens=50).squeeze().tolist()
for pred in preds:
    print(tokenizer.decode(pred), end="")
    if tokenizer.decode(pred) == '.':
        print()
    time.sleep(0.1)

In [ ]:
#torch.save(model.state_dict(), '/kaggle/working/model.pth')